## Dataset analysis for pForest

In [1]:
# Pandas is used for data manipulation
import pandas as pd

In [2]:
# Load the dataset
data = pd.read_csv('../dataset/Tuesday-WorkingHours.pcap_ISCX.csv')
data.head(5)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.3-192.168.10.5-88-49182-6,192.168.10.5,49182,192.168.10.3,88,6,4/7/2017 8:54,640,7,4,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,192.168.10.3-192.168.10.5-88-49183-6,192.168.10.5,49183,192.168.10.3,88,6,4/7/2017 8:54,900,9,4,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,192.168.10.3-192.168.10.5-88-49184-6,192.168.10.5,49184,192.168.10.3,88,6,4/7/2017 8:54,1205,7,4,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,192.168.10.3-192.168.10.5-88-49185-6,192.168.10.5,49185,192.168.10.3,88,6,4/7/2017 8:54,511,7,4,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,192.168.10.3-192.168.10.5-88-49186-6,192.168.10.5,49186,192.168.10.3,88,6,4/7/2017 8:54,773,9,4,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


# Stateful features:
The following are the features we are looking for:
* IAT Packet inter-arrival time (min, max, avg) [25, 24, 22]
* Pkt Len: Packet length (min, max, avg, total) [44, 45, 46, 10]
* Pkt Count: Number of packets [8]
* Flag Count: TCP flag counts (SYN, ACK, PSH, FIN, RST, ECE) [50, 53, 52, 49, 51, 56]
* Duration: Time since first packet [7]

In [3]:
# Remove unwanted columns from dataset:

for i,l in enumerate(list(data.columns.values)):
    if l == " Label" or l == " Flow IAT Max" or l == " Flow IAT Min" or l == ' Total Length of Bwd Packets' or l == ' Total Backward Packets' or l == " Flow IAT Mean" or l == " Min Packet Length" or l == " Max Packet Length" or l == " Packet Length Mean" or l == "Total Length of Fwd Packets" or l == " Total Fwd Packets" or l == " Flow Duration" or l == "FIN Flag Count" or l == " SYN Flag Count" or l == " RST Flag Count" or l == " PSH Flag Count" or l == " ACK Flag Count" or l == " ECE Flag Count":
        continue
    data.drop(l, axis=1, inplace=True)    

In [4]:
data.head(5)

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Flow IAT Mean,Flow IAT Max,Flow IAT Min,Min Packet Length,Max Packet Length,Packet Length Mean,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,ECE Flag Count,Label
0,640,7,4,440,358.0,64.000000,445.0,1.0,0,220,66.500000,0,0,0,1,0,0,BENIGN
1,900,9,4,600,2944.0,75.000000,684.0,1.0,0,1472,253.142857,0,0,0,1,0,0,BENIGN
2,1205,7,4,2776,2830.0,120.500000,777.0,1.0,0,1415,467.166667,0,0,0,1,0,0,BENIGN
3,511,7,4,452,370.0,51.100000,299.0,1.0,0,226,68.500000,0,0,0,1,0,0,BENIGN
4,773,9,4,612,2944.0,64.416667,531.0,1.0,0,1472,254.000000,0,0,0,1,0,0,BENIGN


In [5]:
# Merge together affine columns

print(data.shape)

data['Total Packet Length'] = data['Total Length of Fwd Packets'] + data[' Total Length of Bwd Packets']
data['Packets Number'] = data[' Total Fwd Packets'] +  data[' Total Backward Packets']

# Remove columns not used anymore:
data.drop('Total Length of Fwd Packets', axis=1, inplace=True)   
data.drop(' Total Length of Bwd Packets', axis=1, inplace=True)
data.drop(' Total Fwd Packets', axis=1, inplace=True)
data.drop(' Total Backward Packets', axis=1, inplace=True)


print(data.shape)
data.head(5)

(445909, 18)
(445909, 16)


,Flow Duration,Flow IAT Mean,Flow IAT Max,Flow IAT Min,Min Packet Length,Max Packet Length,Packet Length Mean,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,ECE Flag Count,Label,Total Packet Length,Packets Number
0,640,64.000000,445.0,1.0,0,220,66.500000,0,0,0,1,0,0,BENIGN,798.0,11
1,900,75.000000,684.0,1.0,0,1472,253.142857,0,0,0,1,0,0,BENIGN,3544.0,13
2,1205,120.500000,777.0,1.0,0,1415,467.166667,0,0,0,1,0,0,BENIGN,5606.0,11
3,511,51.100000,299.0,1.0,0,226,68.500000,0,0,0,1,0,0,BENIGN,822.0,11
4,773,64.416667,531.0,1.0,0,1472,254.000000,0,0,0,1,0,0,BENIGN,3556.0,13


In [6]:
# Collapse all non-BENIGN labels to MALIGN
for i,l in enumerate(data[' Label']):
    if l == 'BENIGN':
        continue
    data[' Label'][i] = 'MALIGN'
    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


### Reduce the number of BENIGN flows
The BENIGN flows are the 97%. Training a classifier with a gap as huge as this one would create a strogly biased classifier.
The BENIGN flows are chosen randomly.

In [7]:
from random import seed
from random import randint
# seed random number generator
import time
millis = int(round(time.time() * 1000))
seed(millis)
# generate some integers
randint(0,100)
columns=[" Flow Duration", " Flow IAT Mean", " Flow IAT Max", " Flow IAT Min", " Min Packet Length"," Max Packet Length", " Packet Length Mean", "FIN Flag Count", " SYN Flag Count", " RST Flag Count", " PSH Flag Count", " ACK Flag Count", " ECE Flag Count", " Label", 'Total Packet Length', 'Packets Number']
cut_data = pd.DataFrame(columns=columns)

for i,row in data.iterrows():
    if row[13] == "MALIGN":
        cut_data = cut_data.append(row, ignore_index=True)
    else:
        rnd = randint(0,100)
        if (rnd > 96):
            cut_data = cut_data.append(row, ignore_index=True)
            
print(cut_data.shape)

(31124, 16)


In [8]:
# One-hot encode the data using pandas get_dummies
# making dataframe using get_dummies() 
dummies = cut_data[" Label"].str.get_dummies(" ") 
  
# display 
dummies.head(5) 

,BENIGN,MALIGN
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


In [9]:
dummies.describe()

,BENIGN,MALIGN
count,31124.000000,31124.000000
mean,0.555488,0.444512
std,0.496920,0.496920
min,0.000000,0.000000
25%,0.000000,0.000000
50%,1.000000,0.000000
75%,1.000000,1.000000
max,1.000000,1.000000


#### As we can see, now we have almost the same number of flows for the 2 labels

In [10]:
# Remove Labels from data:
cut_data.drop(' Label', axis=1, inplace=True)

In [11]:
print("Data",cut_data.shape)
print("Dummies", dummies.shape)

Data (31124, 15)
Dummies (31124, 2)


#### Dataset:

In [12]:
#cut_data = cut_data.round(0).astype(int)
cut_data.head(5)

,Flow Duration,Flow IAT Mean,Flow IAT Max,Flow IAT Min,Min Packet Length,Max Packet Length,Packet Length Mean,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,ECE Flag Count,Total Packet Length,Packets Number
0,62,20.666667,60.0,1.0,0,0,0.000000,0,0,0,0,1,0,0.0,4
1,5194037,577115.222200,5005034.0,3.0,0,646,65.727273,0,0,0,1,0,0,723.0,10
2,853,71.083333,628.0,1.0,0,1556,267.428571,0,0,0,1,0,0,3744.0,13
3,1153,82.357143,836.0,0.0,0,1588,392.625000,0,0,0,1,0,0,6282.0,15
4,35,35.000000,35.0,35.0,0,0,0.000000,0,0,0,0,1,0,0.0,2


In [13]:
stateful_data = cut_data.to_numpy()
stateful_dummies = dummies.to_numpy()

#### Save the analysed and reduced dataset into numpy arrays

In [14]:
import numpy as np

np.save("storage/np_data", stateful_data, allow_pickle=True)
np.save("storage/np_dummies", stateful_dummies, allow_pickle=True)